In [1]:
import pandas as pd
import numpy as np

# Step 1: Read the energy data file
DataFrameWithEnergy = pd.read_excel("En_In.xls", skiprows=2, usecols="C:F")
DataFrameWithEnergy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

# Replace "..." with NaN and convert values to numeric type
DataFrameWithEnergy['Energy Supply'] = DataFrameWithEnergy['Energy Supply'].replace("...", np.nan)
DataFrameWithEnergy['Energy Supply per Capita'] = DataFrameWithEnergy['Energy Supply per Capita'].replace("...", np.nan)
DataFrameWithEnergy['Energy Supply'] = pd.to_numeric(DataFrameWithEnergy['Energy Supply'], errors='coerce') * 1_000_000

# Step 3: Rename specific countries
country_renames = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}
DataFrameWithEnergy['Country'] = DataFrameWithEnergy['Country'].replace(country_renames)ы

# Step 4: Remove digits and text in parentheses from country names
DataFrameWithEnergy['Country'] = DataFrameWithEnergy['Country'].str.replace(r'\d+', '', regex=True)
DataFrameWithEnergy['Country'] = DataFrameWithEnergy['Country'].str.replace(r'\s*\(.*\)', '', regex=True).str.strip()

# Set index to country name
DataFrameWithEnergy = DataFrameWithEnergy.set_index('Country')[['Energy Supply', 'Energy Supply per Capita', '% Renewable']]

print("----------------------------------------")

# Check for the presence of specific countries
CheckRightInfo = DataFrameWithEnergy.loc[DataFrameWithEnergy.index.isin(['American Samoa', 'South Korea', 'Bolivia'])]
print(CheckRightInfo)
print("----------------------------------------")

# Read the GDP data file
GPDDataFrame = pd.read_csv("gpd.csv", skiprows=4)

country_renames = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}

GPDDataFrame['Country Name'] = GPDDataFrame['Country Name'].replace(country_renames)
GPDDataFrame = GPDDataFrame.rename(columns={'Country Name': 'Country'}).set_index('Country').loc[:, '2006':'2015']

print(GPDDataFrame.head(1))
print("----------------------------------------")

# Read the Scimagojr data file
ScimagojrDataFrame = pd.read_excel("scimagojr.xlsx")
ScimagojrDataFrame = ScimagojrDataFrame.loc[ScimagojrDataFrame['Rank'] <= 15]
ScimagojrDataFrame = ScimagojrDataFrame.set_index('Country')[['Rank', 'Documents', 'Citable documents', 'Citations',
                                                               'Self-citations', 'Citations per document', 'H index']]

# Merge all DataFrames
combined_df = pd.merge(ScimagojrDataFrame, DataFrameWithEnergy, left_index=True, right_index=True, how='inner')
combined_df = pd.merge(combined_df, GPDDataFrame, left_index=True, right_index=True, how='inner')

print(combined_df.head(3))
print("----------------------------------------")


----------------------------------------
                Energy Supply Energy Supply per Capita % Renewable
Country                                                           
American Samoa            NaN                      NaN    0.641026
Bolivia          3.360000e+08                       32    31.47712
South Korea      1.100700e+10                      221    2.279353
----------------------------------------
         2006  2007  2008  2009          2010  2011  2012  2013  2014  2015
Country                                                                    
Aruba     NaN   NaN   NaN   NaN  2.467704e+09   NaN   NaN   NaN   NaN   NaN
----------------------------------------
                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     127050             126767     597237   
Japan                  3      30504              30287     223024   
Russian Federation     5      18534   

In [2]:
# task 8
def calculateAverageGpd(OurDF):
    GDPcolumns = OurDF.loc[:, '2006':'2015']
    
    averageGDP = GDPcolumns.mean(axis=1)
    
    averageGDPseries = averageGDP.sort_values(ascending=False)
    
    return averageGDPseries

print(calculateAverageGpd(combined_df))

Country
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64


In [27]:
# task 9
def gdpChangeForFifthCountry(OurDF):
    averageGDP = OurDF.loc[:, '2006':'2015'].mean(axis=1)
    
    fifthCountry = averageGDP.nlargest(5).index[-1]
    
    GDPchange = OurDF.loc[fifthCountry, '2015'] - OurDF.loc[fifthCountry, '2006']
    
    return (fifthCountry, GDPchange)

print(gdpChangeForFifthCountry(combined_df))

('Brazil', 474343725980.5698)


In [4]:
# task 10
def maxRenewableEnergyCountry(OurDF):
    maxRenewableCountry = OurDF['% Renewable'].idxmax()
    maxRenewablePercentage = OurDF.loc[maxRenewableCountry, '% Renewable']
    
    return (maxRenewableCountry, maxRenewablePercentage)
print(maxRenewableEnergyCountry(combined_df))

('Brazil', 69.64803)


In [30]:
# task 11
def estimatePopulation(OurDF):
    OurDF['Population Estimate'] = OurDF['Energy Supply'] / OurDF['Energy Supply per Capita']

    OurDF['Population Estimate'] = pd.to_numeric(combined_df['Population Estimate'], errors='coerce')

    OurDF = OurDF.dropna(subset=['Population Estimate'])

    sixthCountry = OurDF.nlargest(6, 'Population Estimate').iloc[-1]

    return (sixthCountry.name, sixthCountry['Population Estimate'])

print(estimatePopulation(combined_df))

('Germany', 80369696.96969697)


In [6]:
# task 12
def calculateCitationPerCapitaCorrelation(OurDF):

    OurDF['Population Estimate'] = OurDF['Energy Supply'] / OurDF['Energy Supply per Capita']

    
    OurDF['Citations per Capita'] = OurDF['Citable documents'] / OurDF['Population Estimate']

    OurDF['Citations per Capita'] = pd.to_numeric(OurDF['Citations per Capita'], errors='coerce')

    OurDF = OurDF.dropna(subset=['Citations per Capita', 'Energy Supply per Capita'])

    correlation = OurDF['Citations per Capita'].corr(OurDF['Energy Supply per Capita'])
    
    return correlation

print(calculateCitationPerCapitaCorrelation(combined_df))

0.8642899768854405


In [7]:
# task 13
def createRenewableIndicator(OurDF):
    median_renewable = OurDF['% Renewable'].median()

    OurDF['Renewable Indicator'] = (OurDF['% Renewable'] >= median_renewable).astype(int)

    return OurDF['Renewable Indicator']

print(createRenewableIndicator(combined_df)) 

Country
China                 1
Japan                 0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: Renewable Indicator, dtype: int32


In [31]:
def summarizePopulationByContinent(OurDF):
    if 'Population Estimate' not in OurDF.columns:
        OurDF['Population Estimate'] = OurDF['Energy Supply'] / OurDF['Energy Supply per Capita']

    ContinentDict = {
        'China': 'Asia',
        'United States': 'North America',
        'Japan': 'Asia',
        'United Kingdom': 'Europe',
        'Russian Federation': 'Europe',
        'Canada': 'North America',
        'Germany': 'Europe',
        'India': 'Asia',
        'France': 'Europe',
        'South Korea': 'Asia',
        'Italy': 'Europe',
        'Spain': 'Europe',
        'Iran': 'Asia',
        'Australia': 'Australia',
        'Brazil': 'South America'
    }
    
    OurDF['Continent'] = OurDF.index.map(ContinentDict)
    
    grouped_df = OurDF.groupby('Continent').agg(
        size=('Rank', 'size'),  
        sum=('Population Estimate', 'sum'),  
        mean=('Population Estimate', 'mean'),  
        std=('Population Estimate', 'std')  
    )
    
    return grouped_df

print(summarizePopulationByContinent(combined_df))

               size           sum          mean           std
Continent                                                    
Asia              5  2.898666e+09  5.797333e+08  6.790979e+08
Australia         1  2.331602e+07  2.331602e+07           NaN
Europe            5  3.940587e+08  7.881174e+07  3.813228e+07
North America     1  3.523986e+07  3.523986e+07           NaN
South America     1  2.059153e+08  2.059153e+08           NaN
